# Customer Segmentation Analysis
## Clustering and Visualization with Hierarchical Department/Class Structure

This notebook demonstrates:
- Generating synthetic retail customer data with hierarchical product structure
- Applying Fuzzy C-Means clustering
- Applying Neural Network clustering
- Visualizing customer segments and their characteristics
- Analyzing department, class, and size preferences by segment

## 1. Setup and Import Libraries

In [ ]:
# Standard libraries
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Custom modules
from customer_segmentation import RetailDataGenerator, get_config
from customer_segmentation.fuzzy_clustering import FuzzyCustomerSegmentation
from customer_segmentation.neural_clustering import NeuralCustomerSegmentation
from customer_segmentation.cluster_enrichment import ClusterEnrichment

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All libraries imported successfully!")

## 2. Load Configuration and Generate Customer Data

In [ ]:
# Load configuration
config = get_config()
print(f"Configuration loaded from: {config.config_path}")

# Initialize data generator
generator = RetailDataGenerator(seed=config.data_generation.random_seed)

# Generate customer data with hierarchical department/class structure
data = generator.generate_customers(
    n_customers=config.data_generation.n_customers,
    segment_probs=list(config.data_generation.segment_probabilities.values())
)

print(f"\n✓ Generated {len(data)} customer records")
print(f"Features ({len(data.columns)}): {list(data.columns)}")
print(f"\nData shape: {data.shape}")
data.head()

## 3. Exploratory Data Analysis

In [ ]:
# Display basic statistics
print("=== Basic Statistics ===")
core_features = ['total_purchases', 'total_revenue', 'avg_order_value', 
                 'recency_days', 'frequency_per_month', 'customer_lifetime_months', 'return_rate']
data[core_features].describe()

In [ ]:
# Visualize RFM distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('RFM Feature Distributions', fontsize=16, fontweight='bold')

# Plot distributions
features_to_plot = ['total_revenue', 'frequency_per_month', 'recency_days', 
                    'avg_order_value', 'total_purchases', 'return_rate']

for idx, feature in enumerate(features_to_plot):
    row = idx // 3
    col = idx % 3
    axes[row, col].hist(data[feature], bins=30, edgecolor='black', alpha=0.7)
    axes[row, col].set_title(feature.replace('_', ' ').title())
    axes[row, col].set_xlabel('Value')
    axes[row, col].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize department and class distributions
dept_cols = [col for col in data.columns if col.startswith('dept_total_value_')]
class_cols = [col for col in data.columns if col.startswith('class_total_value_')]

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Department totals
dept_totals = data[dept_cols].sum()
axes[0].bar(range(len(dept_totals)), dept_totals.values, color='steelblue', edgecolor='black')
axes[0].set_xticks(range(len(dept_totals)))
axes[0].set_xticklabels([col.replace('dept_total_value_', '') for col in dept_cols], rotation=45, ha='right')
axes[0].set_title('Total Sales by Department', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Total Value ($)')
axes[0].grid(axis='y', alpha=0.3)

# Class totals
class_totals = data[class_cols].sum()
axes[1].bar(range(len(class_totals)), class_totals.values, color='coral', edgecolor='black')
axes[1].set_xticks(range(len(class_totals)))
axes[1].set_xticklabels([col.replace('class_total_value_', '') for col in class_cols], rotation=45, ha='right')
axes[1].set_title('Total Sales by Class', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Total Value ($)')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Fuzzy C-Means Clustering

In [ ]:
# Initialize and fit Fuzzy C-Means clustering
fuzzy_config = config.to_dict()['fuzzy_clustering']
fuzzy_model = FuzzyCustomerSegmentation(
    n_clusters=fuzzy_config['n_clusters'],
    m=fuzzy_config['fuzziness_parameter'],
    max_iter=fuzzy_config['max_iterations'],
    error=fuzzy_config['tolerance'],
    seed=fuzzy_config['random_seed']
)

# Store config for feature selection
fuzzy_model.config = config.to_dict()

# Fit and predict
print("Fitting Fuzzy C-Means clustering...")
fuzzy_labels, fuzzy_membership = fuzzy_model.fit_predict(data)

# Add results to dataframe
data['fuzzy_cluster'] = fuzzy_labels

# Display cluster distribution
print(f"\n✓ Fuzzy clustering completed with {fuzzy_config['n_clusters']} clusters")
print("\nCluster distribution:")
print(data['fuzzy_cluster'].value_counts().sort_index())

# Calculate metrics
fuzzy_metrics = fuzzy_model.evaluate(data)
print(f"\nFuzzy Clustering Metrics:")
for metric, value in fuzzy_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
# Display cluster centers
cluster_centers_df = fuzzy_model.get_cluster_centers()
print("\n=== Fuzzy Cluster Centers ===")
cluster_centers_df

## 5. Neural Network Clustering

In [ ]:
# Initialize and fit Neural Network clustering
neural_config = config.to_dict()['neural_clustering']
neural_model = NeuralCustomerSegmentation(
    n_clusters=neural_config['n_clusters'],
    encoding_dim=neural_config['encoding_dim'],
    epochs=neural_config['epochs'],
    batch_size=neural_config['batch_size'],
    seed=neural_config['random_seed']
)

# Store config for feature selection
neural_model.config = config.to_dict()

# Fit and predict
print("Training Neural Network clustering (this may take a moment)...")
neural_labels = neural_model.fit_predict(data, verbose=0)

# Add results to dataframe
data['neural_cluster'] = neural_labels

# Display cluster distribution
print(f"\n✓ Neural clustering completed with {neural_config['n_clusters']} clusters")
print("\nCluster distribution:")
print(data['neural_cluster'].value_counts().sort_index())

# Calculate metrics
neural_metrics = neural_model.evaluate(data)
print(f"\nNeural Clustering Metrics:")
for metric, value in neural_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
# Display neural cluster centers
neural_centers_df = neural_model.get_cluster_centers()
print("\n=== Neural Cluster Centers ===")
neural_centers_df

## 5.5 Gaussian Mixture Models (GMM) Clustering

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

# Initialize GMM clustering
gmm_config = config.to_dict()['fuzzy_clustering']  # Use same config for n_clusters
n_clusters_gmm = gmm_config['n_clusters']

# Prepare features for GMM (using enriched features if enabled)
if gmm_config.get('use_enriched_features', False):
    feature_cols_gmm = gmm_config['features_to_use'] + gmm_config['enriched_features_to_use']
else:
    feature_cols_gmm = gmm_config['features_to_use']

feature_cols_gmm = [col for col in feature_cols_gmm if col in data.columns]

# Standardize features
scaler_gmm = StandardScaler()
X_gmm = data[feature_cols_gmm].values
X_gmm_normalized = scaler_gmm.fit_transform(X_gmm)

# Fit Gaussian Mixture Model
print(f"Fitting Gaussian Mixture Model with {n_clusters_gmm} components...")
gmm_model = GaussianMixture(
    n_components=n_clusters_gmm,
    covariance_type='full',  # Full covariance matrix
    random_state=gmm_config['random_seed'],
    max_iter=200,
    n_init=10
)

# Fit and predict
gmm_labels = gmm_model.fit_predict(X_gmm_normalized)
gmm_proba = gmm_model.predict_proba(X_gmm_normalized)

# Add results to dataframe
data['gmm_cluster'] = gmm_labels

# Display cluster distribution
print(f"\n✓ GMM clustering completed with {n_clusters_gmm} components")
print(f"Converged: {gmm_model.converged_}")
print(f"Number of iterations: {gmm_model.n_iter_}")
print("\nCluster distribution:")
print(data['gmm_cluster'].value_counts().sort_index())

# Calculate metrics
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

gmm_silhouette = silhouette_score(X_gmm_normalized, gmm_labels)
gmm_bic = gmm_model.bic(X_gmm_normalized)
gmm_aic = gmm_model.aic(X_gmm_normalized)
gmm_davies_bouldin = davies_bouldin_score(X_gmm_normalized, gmm_labels)
gmm_calinski = calinski_harabasz_score(X_gmm_normalized, gmm_labels)

gmm_metrics = {
    'silhouette_score': gmm_silhouette,
    'bic': gmm_bic,
    'aic': gmm_aic,
    'davies_bouldin_index': gmm_davies_bouldin,
    'calinski_harabasz_score': gmm_calinski,
    'log_likelihood': gmm_model.score(X_gmm_normalized) * len(X_gmm_normalized),
    'n_clusters': float(n_clusters_gmm)
}

print(f"\nGMM Clustering Metrics:")
for metric, value in gmm_metrics.items():
    if metric in ['bic', 'aic', 'log_likelihood', 'calinski_harabasz_score']:
        print(f"  {metric}: {value:.2f}")
    else:
        print(f"  {metric}: {value:.4f}")

In [ ]:
# Display GMM cluster centers (means)
gmm_centers = pd.DataFrame(
    scaler_gmm.inverse_transform(gmm_model.means_),
    columns=feature_cols_gmm
)
gmm_centers.index = [f'Cluster_{i}' for i in range(n_clusters_gmm)]

print("\n=== GMM Cluster Centers (Means) ===")
# Show core features only for readability
core_features_display = [col for col in gmm_centers.columns if col in core_features]
gmm_centers[core_features_display]

In [ ]:
# Visualize GMM cluster probabilities (soft assignments)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# GMM hard clustering
scatter1 = axes[0].scatter(data['recency_days'], data['total_revenue'], 
                           c=data['gmm_cluster'], cmap='viridis', 
                           alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Recency (days)', fontsize=12)
axes[0].set_ylabel('Total Revenue ($)', fontsize=12)
axes[0].set_title('GMM Clustering (Hard Assignment)', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# GMM soft clustering - show maximum probability
max_proba = gmm_proba.max(axis=1)
scatter2 = axes[1].scatter(data['recency_days'], data['total_revenue'], 
                           c=max_proba, cmap='plasma', 
                           alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Recency (days)', fontsize=12)
axes[1].set_ylabel('Total Revenue ($)', fontsize=12)
axes[1].set_title('GMM Clustering (Assignment Confidence)', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Max Probability')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize GMM probability matrix (sample of customers)
sample_size = min(50, len(data))
sample_indices = np.random.choice(len(data), sample_size, replace=False)

plt.figure(figsize=(12, 8))
sns.heatmap(gmm_proba[sample_indices], 
            annot=False, 
            cmap='YlOrRd', 
            cbar_kws={'label': 'Probability'},
            xticklabels=[f'Cluster {i}' for i in range(n_clusters_gmm)],
            yticklabels=[f'Customer {i}' for i in sample_indices[:20]] + ['...'] * (sample_size - 20) if sample_size > 20 else [f'Customer {i}' for i in sample_indices])

plt.title(f'GMM Cluster Membership Probabilities (Sample of {sample_size} Customers)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Customer', fontsize=12)
plt.tight_layout()
plt.show()

# Show statistics about cluster assignment confidence
avg_max_proba = gmm_proba.max(axis=1).mean()
print(f"\nGMM Assignment Confidence Statistics:")
print(f"  Average maximum probability: {avg_max_proba:.4f}")
print(f"  Customers with >90% confidence: {(gmm_proba.max(axis=1) > 0.9).sum()} ({(gmm_proba.max(axis=1) > 0.9).sum()/len(data)*100:.1f}%)")
print(f"  Customers with <70% confidence: {(gmm_proba.max(axis=1) < 0.7).sum()} ({(gmm_proba.max(axis=1) < 0.7).sum()/len(data)*100:.1f}%)")

## 6. Cluster Enrichment and Segment Naming

In [ ]:
# Enrich fuzzy clusters with meaningful descriptions
enricher = ClusterEnrichment()
enriched_segments = enricher.enrich_clusters(data, 'neural_cluster')

print(f"✓ Enriched {len(enriched_segments)} cluster profiles\n")

# Display segment summaries
for segment in enriched_segments:
    print(f"\n{'='*60}")
    print(f"Cluster {segment['cluster_id']}: {segment['segment_name']}")
    print(f"{'='*60}")
    print(f"\nDescription: {segment['description']}")
    print(f"\nKey Characteristics:")
    print(f"  - Segment Size: {segment['size']} customers ({segment['percentage']:.1f}%)")
    print(f"  - Avg Revenue: ${segment['avg_revenue']:,.2f}")
    print(f"  - Avg Order Value: ${segment['avg_order_value']:.2f}")
    print(f"  - Avg Frequency: {segment['avg_frequency']:.2f} purchases/month")
    print(f"  - Avg Recency: {segment['avg_recency']:.0f} days")

## 7. Visualize Clustering Results

In [ ]:
# RFM scatter plot with all three clustering methods
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Fuzzy clustering
scatter1 = axes[0].scatter(data['recency_days'], data['total_revenue'], 
                           c=data['fuzzy_cluster'], cmap='viridis', 
                           alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Recency (days)', fontsize=12)
axes[0].set_ylabel('Total Revenue ($)', fontsize=12)
axes[0].set_title('Fuzzy C-Means Clustering', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# Neural clustering
scatter2 = axes[1].scatter(data['recency_days'], data['total_revenue'], 
                           c=data['neural_cluster'], cmap='viridis', 
                           alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Recency (days)', fontsize=12)
axes[1].set_ylabel('Total Revenue ($)', fontsize=12)
axes[1].set_title('Neural Network Clustering', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

# GMM clustering
scatter3 = axes[2].scatter(data['recency_days'], data['total_revenue'], 
                           c=data['gmm_cluster'], cmap='viridis', 
                           alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[2].set_xlabel('Recency (days)', fontsize=12)
axes[2].set_ylabel('Total Revenue ($)', fontsize=12)
axes[2].set_title('GMM Clustering', fontsize=14, fontweight='bold')
axes[2].grid(alpha=0.3)
plt.colorbar(scatter3, ax=axes[2], label='Cluster')

plt.tight_layout()
plt.show()

In [ ]:
# Cluster size comparison - all three methods
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Fuzzy cluster distribution
fuzzy_counts = data['fuzzy_cluster'].value_counts().sort_index()
axes[0].bar(fuzzy_counts.index, fuzzy_counts.values, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Cluster', fontsize=12)
axes[0].set_ylabel('Number of Customers', fontsize=12)
axes[0].set_title('Fuzzy C-Means - Cluster Sizes', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(fuzzy_counts.values):
    axes[0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Neural cluster distribution
neural_counts = data['neural_cluster'].value_counts().sort_index()
axes[1].bar(neural_counts.index, neural_counts.values, color='coral', edgecolor='black')
axes[1].set_xlabel('Cluster', fontsize=12)
axes[1].set_ylabel('Number of Customers', fontsize=12)
axes[1].set_title('Neural Network - Cluster Sizes', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(neural_counts.values):
    axes[1].text(i, v + 5, str(v), ha='center', fontweight='bold')

# GMM cluster distribution
gmm_counts = data['gmm_cluster'].value_counts().sort_index()
axes[2].bar(gmm_counts.index, gmm_counts.values, color='mediumseagreen', edgecolor='black')
axes[2].set_xlabel('Cluster', fontsize=12)
axes[2].set_ylabel('Number of Customers', fontsize=12)
axes[2].set_title('GMM - Cluster Sizes', fontsize=14, fontweight='bold')
axes[2].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(gmm_counts.values):
    axes[2].text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 8. Department and Class Preferences by Cluster

In [ ]:
# Department preferences by cluster (using neural clusters)
dept_cols = [col for col in data.columns if col.startswith('dept_total_value_')]
dept_by_cluster = data.groupby('neural_cluster')[dept_cols].mean()

# Rename columns for better display
dept_by_cluster.columns = [col.replace('dept_total_value_', '') for col in dept_by_cluster.columns]

# Create heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(dept_by_cluster.T, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Average Value ($)'}, linewidths=0.5)
plt.title('Department Preferences by Cluster (Neural Clustering)', fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Department', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Class preferences by cluster (using neural clusters)
class_cols = [col for col in data.columns if col.startswith('class_total_value_')]
class_by_cluster = data.groupby('neural_cluster')[class_cols].mean()

# Rename columns for better display
class_by_cluster.columns = [col.replace('class_total_value_', '') for col in class_by_cluster.columns]

# Create heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(class_by_cluster.T, annot=True, fmt='.0f', cmap='Blues', 
            cbar_kws={'label': 'Average Value ($)'}, linewidths=0.5)
plt.title('Class Preferences by Cluster (Neural Clustering)', fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Class', fontsize=12)
plt.tight_layout()
plt.show()

## 9. Size Distribution by Cluster

In [ ]:
# Size distribution by cluster
size_cols = [col for col in data.columns if col.startswith('count_size_') or col in ['count_Baby', 'count_Child']]
size_by_cluster = data.groupby('neural_cluster')[size_cols].mean()

# Rename columns for better display
size_by_cluster.columns = [col.replace('count_size_', '').replace('count_', '') for col in size_by_cluster.columns]

# Create grouped bar chart
fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(size_by_cluster.columns))
width = 0.2

for i, cluster in enumerate(size_by_cluster.index):
    offset = width * (i - len(size_by_cluster.index) / 2)
    ax.bar(x + offset, size_by_cluster.iloc[i], width, 
           label=f'Cluster {cluster}', edgecolor='black', linewidth=0.5)

ax.set_xlabel('Size/Age Category', fontsize=12)
ax.set_ylabel('Average Count', fontsize=12)
ax.set_title('Size/Age Distribution by Cluster (Neural Clustering)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(size_by_cluster.columns, rotation=0)
ax.legend(title='Cluster', fontsize=10)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Cluster Characteristics Radar Chart

In [ ]:
# Create radar chart for cluster characteristics
from math import pi

# Select features for radar chart
features = ['total_revenue', 'frequency_per_month', 'avg_order_value', 'return_rate']
cluster_stats = data.groupby('neural_cluster')[features].mean()

# Normalize to 0-1 scale for better visualization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cluster_stats_normalized = pd.DataFrame(
    scaler.fit_transform(cluster_stats),
    columns=cluster_stats.columns,
    index=cluster_stats.index
)

# Number of variables
num_vars = len(features)
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for cluster in cluster_stats_normalized.index:
    values = cluster_stats_normalized.loc[cluster].values.tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {cluster}')
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels([f.replace('_', ' ').title() for f in features], size=11)
ax.set_ylim(0, 1)
ax.set_title('Cluster Characteristics (Normalized)', size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

## 11. Compare Clustering Methods

In [ ]:
# Compare all three clustering methods
comparison_df = pd.DataFrame({
    'Method': ['Fuzzy C-Means', 'Neural Network', 'Gaussian Mixture'],
    'Silhouette Score': [
        fuzzy_metrics['silhouette_score'], 
        neural_metrics['silhouette_score'],
        gmm_metrics['silhouette_score']
    ],
    'Clusters': [
        fuzzy_config['n_clusters'], 
        neural_config['n_clusters'],
        gmm_metrics['n_clusters']
    ]
})

# Add method-specific metrics
comparison_df['Method Specific Metric'] = [
    f"Partition Coeff: {fuzzy_metrics['partition_coefficient']:.4f}",
    f"Reconstruction Error: {neural_metrics['reconstruction_error']:.4f}",
    f"BIC: {gmm_metrics['bic']:.2f}"
]

print("\n=== Clustering Methods Comparison ===")
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Silhouette Score comparison
x = np.arange(len(comparison_df))
width = 0.5

bars = axes[0].bar(x, comparison_df['Silhouette Score'], width, 
                   color=['steelblue', 'coral', 'mediumseagreen'], 
                   edgecolor='black', linewidth=1.5)

axes[0].set_xlabel('Clustering Method', fontsize=12)
axes[0].set_ylabel('Silhouette Score', fontsize=12)
axes[0].set_title('Clustering Performance Comparison', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(comparison_df['Method'], rotation=15, ha='right')
axes[0].set_ylim(0, max(comparison_df['Silhouette Score']) * 1.2)
axes[0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}', ha='center', va='bottom', fontweight='bold')

# Additional metrics comparison
metrics_data = {
    'Fuzzy': [fuzzy_metrics['partition_coefficient']],
    'Neural': [neural_metrics['reconstruction_error']],
    'GMM': [gmm_metrics['davies_bouldin_index']]
}

x2 = np.arange(3)
axes[1].bar(0, fuzzy_metrics['partition_coefficient'], width, 
            color='steelblue', edgecolor='black', label='Partition Coefficient')
axes[1].bar(1, neural_metrics['reconstruction_error'], width, 
            color='coral', edgecolor='black', label='Reconstruction Error')
axes[1].bar(2, gmm_metrics['davies_bouldin_index'], width, 
            color='mediumseagreen', edgecolor='black', label='Davies-Bouldin Index')

axes[1].set_xlabel('Clustering Method', fontsize=12)
axes[1].set_ylabel('Metric Value', fontsize=12)
axes[1].set_title('Method-Specific Metrics', fontsize=14, fontweight='bold')
axes[1].set_xticks(x2)
axes[1].set_xticklabels(['Fuzzy C-Means', 'Neural Network', 'GMM'], rotation=15, ha='right')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed comparison
print("\n=== Detailed Metrics Comparison ===")
print(f"\nFuzzy C-Means:")
print(f"  Silhouette Score: {fuzzy_metrics['silhouette_score']:.4f}")
print(f"  Partition Coefficient: {fuzzy_metrics['partition_coefficient']:.4f}")
print(f"  Partition Entropy: {fuzzy_metrics['partition_entropy']:.4f}")

print(f"\nNeural Network:")
print(f"  Silhouette Score: {neural_metrics['silhouette_score']:.4f}")
print(f"  Reconstruction Error: {neural_metrics['reconstruction_error']:.4f}")

print(f"\nGaussian Mixture Model:")
print(f"  Silhouette Score: {gmm_metrics['silhouette_score']:.4f}")
print(f"  BIC: {gmm_metrics['bic']:.2f}")
print(f"  AIC: {gmm_metrics['aic']:.2f}")
print(f"  Davies-Bouldin Index: {gmm_metrics['davies_bouldin_index']:.4f} (lower is better)")
print(f"  Calinski-Harabasz Score: {gmm_metrics['calinski_harabasz_score']:.2f} (higher is better)")

## 12. Summary and Key Insights

In [ ]:
print("="*70)
print("CUSTOMER SEGMENTATION ANALYSIS - SUMMARY")
print("="*70)

print(f"\n📊 Data Overview:")
print(f"  - Total Customers: {len(data)}")
print(f"  - Total Features: {len(data.columns)}")
print(f"  - Hierarchical Structure: Departments → Classes")

print(f"\n🎯 Clustering Results:")
print(f"  - Fuzzy C-Means:")
print(f"    • Clusters: {fuzzy_config['n_clusters']}")
print(f"    • Silhouette Score: {fuzzy_metrics['silhouette_score']:.4f}")
print(f"    • Partition Coefficient: {fuzzy_metrics['partition_coefficient']:.4f}")

print(f"\n  - Neural Network:")
print(f"    • Clusters: {neural_config['n_clusters']}")
print(f"    • Silhouette Score: {neural_metrics['silhouette_score']:.4f}")
print(f"    • Reconstruction Error: {neural_metrics['reconstruction_error']:.4f}")

print(f"\n  - Gaussian Mixture Model:")
print(f"    • Clusters: {int(gmm_metrics['n_clusters'])}")
print(f"    • Silhouette Score: {gmm_metrics['silhouette_score']:.4f}")
print(f"    • BIC: {gmm_metrics['bic']:.2f}")
print(f"    • Davies-Bouldin Index: {gmm_metrics['davies_bouldin_index']:.4f}")

print(f"\n💡 Key Insights:")
print(f"  - All three methods identified {fuzzy_config['n_clusters']} distinct customer segments")
print(f"  - Clusters show clear patterns in department/class preferences")
print(f"  - Size/age distributions vary significantly across segments")
print(f"  - Hierarchical product structure enables detailed preference analysis")
print(f"  - GMM provides probabilistic cluster assignments (soft clustering)")
print(f"  - Different methods capture different aspects of customer behavior")

print(f"\n🏆 Best Performing Method:")
silhouette_scores = {
    'Fuzzy C-Means': fuzzy_metrics['silhouette_score'],
    'Neural Network': neural_metrics['silhouette_score'],
    'GMM': gmm_metrics['silhouette_score']
}
best_method = max(silhouette_scores, key=silhouette_scores.get)
print(f"  - {best_method} (Silhouette Score: {silhouette_scores[best_method]:.4f})")

print(f"\n📈 Next Steps:")
print(f"  - Apply findings to marketing campaign targeting")
print(f"  - Develop personalized product recommendations")
print(f"  - Monitor segment migration over time")
print(f"  - Integrate with AI agents for automated customer interactions")
print(f"  - Use GMM probabilities for customer uncertainty analysis")

print("\n" + "="*70)